## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$


In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import xgboost as xgb
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')

In [3]:
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [4]:
df_train['targetA'] = df_train['target'] <= 7.45

In [5]:
repl_list = [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]),
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]), 
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]

# 반복문을 통해 처리합니다.
for c, d, cnt in repl_list:
    df_train[c] = df_train[c].replace(d)
    if (df_train[c].value_counts().sort_index() != cnt).sum() != 0:
        print('error', c, d, cnt)
    print(c, d, cnt)
    df_test[c] = df_test[c].replace(d)

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [6]:
df_test.select_dtypes('object').apply(lambda x: set(x.unique()) - set(df_train[x.name].unique()))

cat0    {}
cat1    {}
cat2    {}
cat3    {}
cat4    {}
cat5    {}
cat6    {}
cat7    {}
cat8    {}
cat9    {}
dtype: object

In [7]:
from scipy.stats import norm
mu_A, std_A = 6.769, 0.616
mu_B, std_B = 8.123, 0.527

df_train_sub = df_train.loc[
    (df_train['target'] > norm.ppf(0.99, loc=mu_A, scale=std_A)) |
    (df_train['target'] < norm.ppf(0.01, loc=mu_B, scale=std_B))
]

In [8]:
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
ct_xgb = ColumnTransformer([
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)]),
    ('ohe', OneHotEncoder(handle_unknown='ignore'), ['cat{}'.format(i) for i in range(10)])
])
clf_xgb = xgb.XGBClassifier(
    max_depth=2, # 트리의 최대 깊이: 2
    reg_alpha=0.1, # L1 규제: 0.1
    reg_lambda=0.1, # L2 규제: 0.1
    colsample_bytree=0.25, # 트리 당 컬럼 샘플링 비율: 0.25
    n_estimators=500, # 트리의 수: 500
    random_state=123
)
clf_xgb = make_pipeline(ct_xgb, clf_xgb)
X_xgb = ['cont{}'.format(i) for i in range(14)] + ['cat{}'.format(i) for i in range(10)]
clf_xgb.fit(df_train_sub[X_xgb], df_train_sub['targetA'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pt', 'passthrough',
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('ohe',
                                                  OneHotEncoder(categorical_features=None,
                                                                categori...
                 XGBClassifier(base_score=0.5, booster='gbtree',
         

In [9]:
df_train['targetA_prob'] = clf_xgb.predict_proba(df_train[X_xgb])[:, 1]
df_test['targetA_prob'] = clf_xgb.predict_proba(df_test[X_xgb])[:, 1]

In [10]:
# 문제 4에서 뽑아낸 파생변수를 만들어봅니다.
q = [i * 0.01 for i in range(101)]
for i in range(0, 14):
    col = 'cont{}'.format(i)
    col_q = col + '_q' 
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_cat = pd.cut(df_train[col], bins=q_val, right=True)
    q_mean = df_train.groupby(q_cat)['target'].mean()
    df_train[col_q] = q_cat.map(q_mean).astype('float')
    df_test[col_q] = \
                    pd.cut(df_test[col], bins=q_val, right=True).map(q_mean).astype('float')

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
cv = KFold(n_splits=5, shuffle=True, random_state=123)

In [12]:
from sklearn.linear_model import LinearRegression
ct_lr = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob'])
])
X_lr = ['cat{}'.format(i) for i in range(10)] + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']
reg_lr = make_pipeline(ct_lr, LinearRegression())
scores_ = cross_val_score(reg_lr, df_train[X_lr], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_ = np.sqrt(-scores_)
score_ = np.mean(scores_)
scores_, score_

(array([0.84207099, 0.84352739, 0.84673595, 0.84073641, 0.83766386]),
 0.8421469216828141)

## 선형모델을 사용할 때 유의 사항

Unknown 범주값에 대해서 OneHotEncoder 에서 handle_unknown='ignore'로 하면,

그 변수에 대해서는 모든 값이 0이 가변수를 반환합니다. (Unknown 범주값: Test 셋에서 Train에 나오지 않는 범주값을 일컫습니다.)

하지만, OneHotEncoder를 handle_unknown='ignore'로 하면 drop='first'를 사용할 수 없습니다.

(1.0 이후 버젼은 허용합니다. 쩝...)

선형 모델을 사용하지 않으면 문제 없습니다. 

drop='first'를 사용하지 않으면, 가변수 간에 완전한 다중공선성을 갖게 되는데요,

선형 모델의 계수(coefficient)가 큰 값을 지니게 됩니다. 

따라서, 불안정한 모델이 만들어지게 되고, 

학습에서 등장하지 않은 경우(Ex 가변수가 모두 0)에 대해서는 전혀 엉뚱한 값이 나오게 됩니다.

선형 모델을 사용하고자 하고, 사용하고자 하는 변수가 그렇다면.

이에 대한 대처법 test에 train에 나오지 않는 범주값이 나올 경우 대체 시켜줍니다. 

   Ex) 가장 많이 나온 범주

In [13]:
reg_lr.fit(df_train[X_lr], df_train['target'])
pd.DataFrame(
    reg_lr.predict(df_test[X_lr]),
    columns=['target'],
    index=df_test.index
).to_csv('answer6.csv')

In [14]:
# 회귀계수가 말도 안되게 큰게 있을지고 확인해 보시면 좋습니다.
reg_lr[1].coef_, reg_lr[1].intercept_

(array([-0.0388694 ,  0.03764582,  0.06233596,  0.01982507,  0.00640983,
         0.08562987,  0.00846555,  0.05141216,  0.02535578,  0.06141578,
         0.14500394,  0.00692817, -0.00232896, -0.02088899,  0.02644641,
         0.01319598,  0.01820396,  0.00130193, -0.01233047,  0.04228771,
         0.06995946,  0.03070072,  0.00263135, -0.00412637,  0.02514557,
         0.06184112,  0.03042122,  0.0329391 ,  0.01639208,  0.22146555,
         0.36151574,  0.56243077,  0.32152849,  0.5638488 ,  0.28192552,
         0.45796113,  0.52076853,  0.32273993,  0.30980146,  0.2535911 ,
         0.20738769,  0.10024969,  0.23161635, -1.13442501]),
 -27.19109449333468)

In [15]:
# 정답셋에 대한 성능을 봅니다.
mean_squared_error(df_ans['target'], reg_lr.predict(df_test[X_lr])) ** 0.5

0.8480552830027995

In [16]:
# 문제3에서 만들었던 PCA를 적용해봅니다.
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
ct_lr_2 = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), ['cat{}'.format(i) for i in range(10)]),
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=3)), ['cont0', 'cont5', 'cont8', 'cont9', 'cont12']),
    ('pt', 'passthrough', ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob'])
])
X_lr_2 = ['cat{}'.format(i) for i in range(10)] + ['cont{}_q'.format(i) for i in range(14)]\
    + ['cont0', 'cont5', 'cont8', 'cont9', 'cont12', 'targetA_prob']
reg_lr_2 = make_pipeline(ct_lr_2, LinearRegression())
scores_ = cross_val_score(reg_lr_2, df_train[X_lr_2], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_ = np.sqrt(-scores_)
score_ = np.mean(scores_)
scores_, score_

(array([0.84199133, 0.84347491, 0.84660574, 0.84084008, 0.83760069]),
 0.8421025512385348)

**Q: 범주형 변수들의 차원을 낮추는 방법이 있을까요?**

**A: TruncatedSVD를 사용해 볼만 합니다.**

sklearn에서 제공하는 문서를 인용합니다.

**Init signature**

TruncatedSVD(
    n_components=2,
    algorithm='randomized',
    n_iter=5,
    random_state=None,
    tol=0.0,
)

**Docstring**

Dimensionality reduction using truncated SVD (aka LSA).

This transformer performs linear dimensionality reduction by means of
truncated singular value decomposition (SVD). Contrary to PCA, this
estimator does not center the data before computing the singular value
decomposition. This means it can work with scipy.sparse matrices
efficiently.

In particular, truncated SVD works on term count/tf-idf matrices as
returned by the vectorizers in sklearn.feature_extraction.text. In that
context, it is known as latent semantic analysis (LSA).

mean centering을 하지 않기 때문에 sparse한 셋에 대해서 효율적으로 작동합니다. 

Text mining 기법(LSA)에서 활용하는 방법인데요, 가변수는 sparse한 특징이 있기 때문에

잘 맞을 꺼라 판단됩니다. 특히 범주의 수준이 크면 용이하리라 생각됩니다.

In [114]:
# Truncated SVD로 모델을 만들어 봅니다.

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
ct_lr_3 = ColumnTransformer([
    ('ohe', make_pipeline(OneHotEncoder(drop='first'), TruncatedSVD(n_components=10)), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob'])
])
X_lr_3 = ['cat{}'.format(i) for i in range(10)] + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']
reg_lr_3 = make_pipeline(ct_lr_3, LinearRegression())
scores_ = cross_val_score(reg_lr_3, df_train[X_lr_3], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_ = np.sqrt(-scores_)
score_ = np.mean(scores_)
scores_, score_

(array([0.84213783, 0.84385401, 0.84688552, 0.84101823, 0.83797625]),
 0.8423743687616456)

In [112]:
# XGBoost 모델 만들어 봅니다.
# 실행시간이 깁니다. 
import xgboost as xgb

ct_xgb = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)] + ['targetA_prob'])
])
X_xgb = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)] + ['targetA_prob']
reg_xgb = make_pipeline(ct_xgb, xgb.XGBRegressor(max_depth=2, n_estimators=500, colsample_bytree=0.25, random_state=123))
scores_ = cross_val_score(reg_xgb, df_train[X_xgb], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_ = np.sqrt(-scores_)
score_ = np.mean(scores_)
scores_, score_

(array([0.84321514, 0.84458143, 0.84735953, 0.84161917, 0.83843339]),
 0.8430417333299711)

In [115]:
# 4개의 모델을 앙상블해봅니다.
# 실행시간이 깁니다.
from sklearn.ensemble import VotingRegressor
reg_vt = VotingRegressor([
    ('lr', reg_lr),
    ('lr_2', reg_lr_2),
    ('lr_3', reg_lr_3),
    ('xgb', reg_xgb)
])
X_vt = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]\
    + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']
scores_ = cross_val_score(reg_vt, df_train[X_vt], df_train['target'], cv=cv, scoring='neg_mean_squared_error')
scores_ = np.sqrt(-scores_)
score_ = np.mean(scores_)
scores_, score_

(array([0.84183269, 0.84334553, 0.84639468, 0.84054509, 0.83742355]),
 0.8419083077583673)

In [116]:
# 앙상블 모델을 최종제출 합니다.
reg_vt.fit(df_train[X_vt], df_train['target'])
pd.DataFrame(
    reg_vt.predict(df_test[X_vt]),
    columns=['target'],
    index=df_test.index
).to_csv('answer6.csv')

In [118]:
# 정답셋에 대한 성능을 봅니다.
mean_squared_error(df_ans['target'], reg_vt.predict(df_test[X_vt])) ** 0.5

0.847476481802959

**여러분 3일간 고생 많으셨습니다.**

**더 높은 단계에서 다시 만나는 날이 오기를 기다리겠습니다.**

**감사합니다.**
